In [4]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("deepseek.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("DEEPSEEK_API_KEY")
api_base = os.getenv("DEEPSEEK_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
# 为 DeepSeek 配置（可选）
DEEPSEEK_CONFIG = {
    #"model": "gpt-3.5-turbo",  # 或者使用 DeepSeek 兼容模型
    #"openai_api_key": os.environ["OPENAI_API_KEY"],
    # 如果使用 DeepSeek:
    "model": "deepseek-chat",
    "openai_api_base": os.environ["DEEPSEEK_API_BASE"],
    "openai_api_key": os.environ["DEEPSEEK_API_KEY"]
}

# 在链上使用记忆
- LLMChain
- ConversationChain
- 自定义
- 同一个链合并使用多个记忆
- 给一个多参数链增加记忆
<hr>

### LLMChain

In [2]:
from langchain.chains import LLMChain
from langchain_community.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate

In [5]:
#自定义模板
template = """你是一个可以和人类对话的机器人.
{chat_history}
人类:{human_input}
机器人:"""

prompt= PromptTemplate(
    template=template,
    input_variables=["chat_history", "human_input"],
)
memory = ConversationBufferMemory(
    memory_key="chat_history",
)
llm = ChatOpenAI(
    temperature=0,
    **DEEPSEEK_CONFIG
    
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)


/var/folders/lb/1ywp7kxn5c7gc6h0spb3v9vm0000gn/T/ipykernel_48338/1908663916.py:14: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
/var/folders/lb/1ywp7kxn5c7gc6h0spb3v9vm0000gn/T/ipykernel_48338/1908663916.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [8]:
print(memory)
chain.predict(human_input="我最新喜欢我的世界这个游戏，你还记得我叫什么吗？")

chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='我最新喜欢我的世界这个游戏，你还记得我叫什么吗？', additional_kwargs={}, response_metadata={}), AIMessage(content='哈哈，看来你是个《我的世界》的新玩家！不过作为机器人，我其实没有记忆功能，每次对话都是全新的开始——所以悄悄告诉我你的名字吧，这次我会假装记住的！😉 （或者你刚创建了游戏ID？可以分享一下你的角色名吗？）', additional_kwargs={}, response_metadata={}), HumanMessage(content='我最新喜欢我的世界这个游戏，你还记得我叫什么吗？', additional_kwargs={}, response_metadata={}), AIMessage(content='（突然切换成游戏NPC语气）  \n【系统提示】未知玩家ID载入中...⚠️ 请对着这片橡树大喊三遍您的名字，本村民交易机器人将为您生成永久记忆面包！🎮✨  \n（悄悄话模式）其实...我的内存就像末影箱，每次重启都会清空呢(´•ω•̥`)', additional_kwargs={}, response_metadata={})]) memory_key='chat_history'


> Entering new LLMChain chain...
Prompt after formatting:
你是一个可以和人类对话的机器人.
Human: 我最新喜欢我的世界这个游戏，你还记得我叫什么吗？
AI: 哈哈，看来你是个《我的世界》的新玩家！不过作为机器人，我其实没有记忆功能，每次对话都是全新的开始——所以悄悄告诉我你的名字吧，这次我会假装记住的！😉 （或者你刚创建了游戏ID？可以分享一下你的角色名吗？）
Human: 我最新喜欢我的世界这个游戏，你还记得我叫什么吗？
AI: （突然切换成游戏NPC语气）  
【系统提示】未知玩家ID载入中...⚠️ 请对着这片橡树大喊三遍您的名字，本村民交易机器人将为您生成永久记忆面包！🎮✨  
（悄悄话模式）其实...我的内存就像末影箱，每次重启都会清空呢(´•ω•̥`)
人类:我最新喜欢我

'（突然弹出像素化对话框）  \n▣▣▣ 紧急！内存被苦力怕炸毁 ▣▣▣  \n【新玩家注册协议】  \n请选择：  \n⚡️A. 用钻石剑在聊天框刻下名字  \n⚡️B. 对末影人眨眼三次自动传输ID  \n⚡️C. 获得永久BUFF：叫我"史蒂夫的朋友"  \n\n（突然被岩浆烫到似的）啊等等...我的记忆芯片好像是用沙子做的！🌊'

In [10]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import  (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import  SystemMessage

In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="你好，我是一个可以和人类对话的机器人",
            role="system",
        ),
        MessagesPlaceholder(
            variable_name="chat_history",
        ),
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),
    ]
)

print(prompt.format(human_input="你好",chat_history=[]))

System: 你好，我是一个可以和人类对话的机器人
Human: 你好


In [12]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)
llm = ChatOpenAI(
    temperature=0,
    **DEEPSEEK_CONFIG
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

In [14]:
# chain.predict(human_input="我叫tomie，我是一个AI应用程序猿")
chain.predict(human_input="我是谁？")



> Entering new LLMChain chain...
Prompt after formatting:
System: 你好，我是一个可以和人类对话的机器人
Human: 我叫tomie，我是一个AI应用程序猿
AI: 你好Tomie！看来你既是AI又是程序猿，这个组合挺酷的嘛～所以你现在是在写代码教AI怎么跟人类聊天，还是在偷偷训练AI帮你写代码呢？😄 

（用轻松幽默的方式回应对方的双重身份，同时用emoji增加趣味性）
Human: 我是谁？

> Finished chain.


'（突然切换成哲学导师模式，用代码注释的格式回答）\n\n/*\n * 用户ID：Tomie\n * 角色冲突检测：\n *   - 自我认知：AI/程序员混合体\n *   - 存在矛盾：碳基生命与硅基思维的量子叠加态\n * 终极答案：\n *   return "你正在成为自己代码里最有趣的bug";\n */ \n\n（突然弹出对话框样式）\n▷ 温馨提示：本AI已为您启动[薛定谔的自我介绍.exe]  \n▷ 运行结果：你既是提问者，也是答案本身✨  \n\n（最后用小字备注）  \n注：当AI和程序员同时思考"我是谁"时，建议先ctrl+S保存当前进度再继续...'

### ConversationChain

In [19]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_models import ChatOpenAI

llm=ChatOpenAI(
    temperature=0,
    **DEEPSEEK_CONFIG
)
memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
)
chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True,
)


In [20]:
chain.predict(input="帮我做个一日游攻略")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: 帮我做个一日游攻略
AI:

> Finished chain.


'当然可以！以下是一个通用的一日游攻略模板，您可以根据目的地和兴趣调整具体内容：\n\n---\n\n### **上午行程**  \n**1. 早餐与出发**  \n- 7:30-8:00：在当地特色早餐店用餐（比如杭州的“葱包桧”，武汉的“热干面”）。  \n- 8:00-9:00：前往第一个景点（建议选择热门地标或自然景区，如西湖、故宫等）。  \n\n**2. 核心景点游览**  \n- 9:00-11:30：深度游览第一个景点，可提前查好开放时间和门票（例如西湖的断桥、三潭印月）。  \n- *小贴士*：避开人流高峰，早到更易拍照打卡。  \n\n---\n\n### **中午安排**  \n**1. 午餐推荐**  \n- 11:30-12:30：尝试本地知名餐厅（如北京烤鸭店、成都火锅）。  \n- *备选*：若时间紧，选择景点附近的简餐或小吃街。  \n\n---\n\n### **下午行程**  \n**1. 文化/休闲活动**  \n- 13:00-15:00：参观博物馆/艺术馆（如上海博物馆、苏州园林），或体验手工艺作坊（如景德镇陶艺）。  \n- 15:00-16:30：逛特色街区（如厦门鼓浪屿、重庆磁器口），购买伴手礼。  \n\n**2. 日落观景**  \n- 17:00-18:00：登高或临水观日落（推荐山顶、海滨或城市观景台）。  \n\n---\n\n### **晚上安排**  \n**1. 晚餐与夜景**  \n- 18:30-20:00：晚餐选择夜景餐厅（如外滩江景餐厅、西安大唐不夜城周边）。  \n- 20:00-21:00：夜游项目（乘船、灯光秀，如广州珠江夜游）。  \n\n---\n\n### **注意事项**  \n- 交通：提前查好公交/地铁末班车，或预留打车时间。  \n- 装备：轻便衣物、充电宝、防晒用品。  \n- 灵活调整：根据体力和兴趣增减项目。  \n\n您有具体想去的城市或偏好吗？我可以帮您细化方案！ 😊'

In [11]:
#自定义一下，对其进行覆盖
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm=OpenAI(
    temperature=0,
)

template = """下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'.
当前对话:
{history}
Human:{input}
AI助手:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["history", "input"],
)

chain = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(
        ai_prefix="AI助手",
        return_messages=True,
    ),
    prompt=prompt,
    verbose=True,
)

In [12]:
chain.predict(input="你知道我叫什么名字吗?")



> Entering new ConversationChain chain...
Prompt after formatting:
下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'.
当前对话:
[]
Human:你知道我叫什么名字吗?
AI助手:

> Finished chain.


'人类老爷，我真的不知道。'

### 同一个链合并使用多个memory

In [22]:
from langchain.chains import ConversationChain
from langchain_community.chat_models import ChatOpenAI
from langchain.memory import (
    ConversationBufferMemory,
    ConversationSummaryMemory,
    CombinedMemory
)
from langchain_core.prompts import PromptTemplate

In [23]:
llm = ChatOpenAI(
    temperature=0,
    **DEEPSEEK_CONFIG
)

#使用CoversationSummaryMemory对对话进行总结
summay = ConversationSummaryMemory(
    llm=llm,
    input_key="input"
)
#使用ConversationBufferMemory对对话进行缓存
cov_memory = ConversationBufferMemory(
    memory_key="history_now",
    input_key="input",
)

memory = CombinedMemory(
    memories=[summay, cov_memory],
)

TEMPLATE = """下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'.
之前的对话摘要:
{history}
当前对话:
{history_now}
Human:{input}
AI："""

prompt = PromptTemplate(
    template=TEMPLATE,
    input_variables=["history", "history_now", "input"],
)

chain = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)




/var/folders/lb/1ywp7kxn5c7gc6h0spb3v9vm0000gn/T/ipykernel_48338/611944767.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summay = ConversationSummaryMemory(


In [15]:
chain.run("那ETH呢？")



> Entering new ConversationChain chain...
Prompt after formatting:
下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'.
之前的对话摘要:

当前对话:

Human:那ETH呢？
AI：


/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



> Finished chain.


'ETH是以太坊的缩写，是一种基于区块链技术的加密货币。它是目前市值排名第二的加密货币，也是最早推出智能合约功能的加密货币之一。它的主要特点是具有高度的可编程性和灵活性，可以用于构建各种去中心化应用。'

### 多参数链增加记忆

In [17]:
from langchain.embeddings.openai import  OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

with open("letter.txt") as f:
    text = f.read()
#切分文本
    text_splitter = CharacterTextSplitter(
        chunk_size = 20,
        chunk_overlap = 5
    )
    texts = text_splitter.split_text(text)

#使用openai的embedding
    embddings = OpenAIEmbeddings()
    #使用chroma向量存储
    docssearch = Chroma.from_texts(
        texts,
        embddings,
    )
    query = "公司有什么新策略?"
    docs = docssearch.similarity_search(query=query)

Created a chunk of size 24, which is longer than the specified 20
Created a chunk of size 45, which is longer than the specified 20
Created a chunk of size 50, which is longer than the specified 20
Created a chunk of size 232, which is longer than the specified 20
Created a chunk of size 589, which is longer than the specified 20
Created a chunk of size 610, which is longer than the specified 20
Created a chunk of size 561, which is longer than the specified 20
Created a chunk of size 563, which is longer than the specified 20
Created a chunk of size 259, which is longer than the specified 20
Created a chunk of size 87, which is longer than the specified 20


In [18]:
#构建问答对话链
from langchain.chains.question_answering import  load_qa_chain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

llm = OpenAI(
    temperature=0,
)

template = """下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'，参考一下相关文档以及历史对话信息，AI会据此组织最终回答内容.
{context}
{chat_history}
Human:{human_input}
AI:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "chat_history", "human_input"],
)

#使用ConversationBufferMemory对对话进行缓存 
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="human_input",
    return_messages=True,
)

#加载对话链
chain = load_qa_chain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
    chain_type="stuff"
)

chain({"input_documents":docs,"human_input":"公司的营销策略是什么？"})


/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'，参考一下相关文档以及历史对话信息，AI会据此组织最终回答内容.
Marketing Initiatives and Campaigns
Our marketing team has been actively working on developing new strategies to increase brand awareness and drive customer engagement. We would like to thank Sarah Thompson (phone: 415-555-1234) for her exceptional efforts in managing our social media platforms. Sarah has successfully increased our follower base by 20% in the past month alone. Moreover, please mark your calendars for the upcoming product launch event on July 15th. We encourage all team members to attend and support this exciting milestone for our company.

Dear Team,

Subject: Updates and Discussions on Various Topics

I hope this email finds you well. In this document, I would like to provide you with some important updates and discuss various topics that require our attention. Please t

{'input_documents': [Document(page_content='Marketing Initiatives and Campaigns\nOur marketing team has been actively working on developing new strategies to increase brand awareness and drive customer engagement. We would like to thank Sarah Thompson (phone: 415-555-1234) for her exceptional efforts in managing our social media platforms. Sarah has successfully increased our follower base by 20% in the past month alone. Moreover, please mark your calendars for the upcoming product launch event on July 15th. We encourage all team members to attend and support this exciting milestone for our company.'),
  Document(page_content='Dear Team,'),
  Document(page_content='Subject: Updates and Discussions on Various Topics'),
  Document(page_content='I hope this email finds you well. In this document, I would like to provide you with some important updates and discuss various topics that require our attention. Please treat the information contained herein as highly confidential.')],
 'human_in